In [28]:
from pathlib import Path
from PIL import Image, ImageDraw
import json

def split_battle_spot(file_name: str):
  # Split the battle spots (user and opponents battle spot)
  path_name = Path(file_name)
  battle_spot = Image.open(file_name)
  crop_offset = 0
  field_name = path_name.stem
  if field_name == "cynthia":
    crop_offset = 1
  elif field_name in ["aaron", "bertha", "flint", "lucian"]:
    crop_offset = 2

  battle_spot_left = battle_spot.crop((0, battle_spot.height / 2 + (crop_offset), 256, battle_spot.height))
  battle_spot_right = battle_spot.crop((0, 0, 256, battle_spot.height / 2 + (crop_offset)))
  
  return battle_spot_left, battle_spot_right

def generate_shadow(img: Image, center: tuple[int, int], widths: list[int], doubled_center: bool):
    """Draws an ellipsis shadow on image."""
    img_draw = ImageDraw.Draw(img)
    
    for i, w in enumerate(widths):
        x1 = center[0] - (w / 2)
        x2 = center[0] + (w / 2)
        y1 = center[1] - i if doubled_center == False else center[1] - i - 1
        y2 = center[1] + i
        img_draw.line(xy=[(x1, y1), (x2, y1)], fill=(98, 85, 101, 255), width=1)
        img_draw.line(xy=[(x1, y2), (x2, y2)], fill=(98, 85, 101, 255), width=1)
    
    return img

def generate_encounter(sprite_file: str, field_type: str, battle_spot_type: str, x: int, y: int, center: tuple[int, int], widths: list[int], doubled_center: bool) -> Image:
  """Generates an recreation of a Pokemon Encounter"""
  sprite_dir = "../assets/PokemonGen4/sprites"
  rand_field = Image.open(f"../assets/fields/{field_type}.png")
  rand_spot_left, rand_spot_right = split_battle_spot(f"../assets/battle_spots/{battle_spot_type}.png")
  dp_sprite = Image.open(sprite_file)

  rand_field.paste(im=rand_spot_left, box=(-64, 120), mask=rand_spot_left) # Add left spot to field
  rand_field.paste(im=rand_spot_right, box=(64, 64), mask=rand_spot_right) # Add right spot to field
  rand_field = generate_shadow(img=rand_field, center=center, widths=widths, doubled_center=doubled_center) # Add shadow to right spot
  rand_field.paste(im=dp_sprite, box=(x, y), mask=dp_sprite) # Add Pokemon to right spot

  return rand_field

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os

os.makedirs("../dataset", exist_ok=True)

with open("encounter_info.json") as f:
    coordinates_dict = json.load(f)

with open("encounter_shadows.json") as f:
    shadow_dict = json.load(f)

with open("field_map.json") as f:
    field_map = json.load(f)

# Variables needed to generate encounters
f_bs_pairs = []
weights = []

for key in field_map.keys():
    for bs in field_map[key]["battle_spots"]:
        f_bs_pairs.append((key, bs))
        weights.append(field_map[key]["weight"])

weights = np.array(weights, dtype=float)
weights /= weights.sum()

rng = np.random.default_rng()
sprite_dir = "../assets/PokemonGen4/sprites"

for i in range(1): # Pokemon 1 - 493
    pkmn_index = i + 1
    x, y = coordinates_dict[str(pkmn_index)]["x"], coordinates_dict[str(pkmn_index)]["y"]
    center = shadow_dict[str(pkmn_index)]["center"]
    widths = shadow_dict[str(pkmn_index)]["widths"]
    dc = shadow_dict[str(pkmn_index)]["doubled_center"]
    sprite_files = list(Path(sprite_dir + f"/{pkmn_index}").glob("*.png"))
    sprite_files_len = len(sprite_files)
    
    for i in range(2): # Generate 25 images for normal, then shiny
        rand_f_bs_pairs = rng.choice(a=f_bs_pairs, size=24, replace=False, p=weights, axis=0, shuffle=False)
        print(rand_f_bs_pairs)
        for i, f_bs_pairs in enumerate(rand_f_bs_pairs):
            field_type, battle_spot = f_bs_pairs
            encounter_img = generate_encounter(
                    sprite_file=str(sprite_files[i % sprite_files_len]), 
                    field_type=field_type,
                    battle_spot_type=battle_spot,
                    x=x,
                    y=y,
                    center=center,
                    widths=widths,
                    doubled_center=dc
            )
            encounter_img.save(f"../dataset/{field_type}_{battle_spot}.png")


[['surf_day' 'water_day']
 ['snow_night' 'grass_night']
 ['forest_day' 'mud_day']
 ['field_night' 'grass_night']
 ['field_afternoon' 'field_afternoon']
 ['rocky_night' 'rocky_night']
 ['surf_night' 'water_night']
 ['forest_afternoon' 'mud_afternoon']
 ['battle_factory' 'battle_factory']
 ['cave_3' 'cave']
 ['field_night' 'wet_path_night']
 ['cave_1' 'cave']
 ['surf_afternoon' 'water_afternoon']
 ['forest_day' 'grass_day']
 ['cave_2' 'cave']
 ['forest_night' 'grass_night']
 ['field_day' 'wet_path_day']
 ['forest_night' 'mud_night']
 ['indoor_3' 'indoor']
 ['snow_night' 'snow_night']
 ['snow_afternoon' 'grass_afternoon']
 ['rocky_night' 'grass_night']
 ['snow_day' 'grass_day']
 ['forest_afternoon' 'grass_afternoon']]


FileNotFoundError: [Errno 2] No such file or directory: '../assets/battle_spots/field_afternoon.png'

: 